# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

### State Representation
[W1/W2] W1:0..c1, W2:0..c2

### Initial State
[0,0]

### Operators (Name, Preconditions, Effects, Cost)
- Fill: 
  - Preconditions: The container is not full.
  - Effects: The container is full.
  - Cost: 1
- Empty:
  - Preconditions: The container is not empty.
  - Effects: The container is empty.
  - Cost: 1
- Pour:
  - Preconditions: The source container is not empty and the target container is not full.
  - Effects: The source container is empty and the target container is full.
  - Cost: 1

### Objective Test
[n, _]




What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

### Where c1 and c2 are whole numbers, the size of the state space is (c1+1)*(c2+1).

Solve the problem by hand, using tree search. What solutions have you found?

<img src="./imgs/tree.png" alt="Tree image" />

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [1]:
from typing import Optional

class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    IDEAL_BUCKET_1 = 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [2]:
# emptying the first bucket
def empty1(state: BucketState)->Optional[BucketState]:
    """Empty the first bucket

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the bucket is already empty
    """
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state: BucketState)->Optional[BucketState]:
    """Empty the second bucket

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the bucket is already empty
    """
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None
    

# Fill the first bucket
def fill1(state: BucketState)->Optional[BucketState]:
    """Fill the first bucket

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the bucket is already full
    """
    if state.b1 < BucketState.c1:
        return BucketState(BucketState.c1, state.b2)
    return None


# Fill the second bucket
def fill2(state: BucketState)->Optional[BucketState]:
    """Fill the second bucket

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the bucket is already full
    """
    if state.b2 < BucketState.c2:
        return BucketState(state.b1, BucketState.c2)
    return None

# Pour from the first bucket to the second and fill the second
def pour12_fill2(state: BucketState)->Optional[BucketState]:
    """Pour from the first bucket to the second and fill the second

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the first bucket is empty or the second is full
    """
    if state.b1 > 0 and state.b2 < BucketState.c2 and state.b1 >= BucketState.c2 - state.b2:
        diff = abs(BucketState.c2 - state.b2)
        return BucketState(state.b1 - diff, BucketState.c2)
    return None


# Pour from first to second and empty 1
def pour12_empty1(state: BucketState)->Optional[BucketState]:
    """Pour from first to second and empty 1

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the first bucket is empty or the second is full
    """
    if state.b1 > 0 and state.b2 < BucketState.c2 and state.b1 < BucketState.c2-state.b2:
       
        return BucketState(0, state.b1 + state.b2)
    return None


# Pour second to first and fill 1
def pour21_fill1(state: BucketState)->Optional[BucketState]:
    """Pour second to first and fill 1

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the second bucket is empty or the first is full
    """
    if state.b2 > 0 and state.b1 < BucketState.c1 and state.b2 >= BucketState.c1 - state.b1:
        diff = abs(BucketState.c1 - state.b1)
        return BucketState(BucketState.c1, state.b2 - diff)
    return None


# Pour second to first and empty 2
def pour21_empty2(state: BucketState)->Optional[BucketState]:
    """Pour second to first and empty 2

    Args:
        state (BucketState): State of the buckets

    Returns:
        BucketState|None: New state of the buckets or None if the second bucket is empty or the first is full
    """
    if state.b2 > 0 and state.b1 < BucketState.c1 and state.b2 < BucketState.c1-state.b1:
        return BucketState(state.b1 + state.b2, 0)
    return None


The following function will aggregate all states that can be generated from a given one:

In [3]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [4]:
s = BucketState(0, 0)
s = fill1(s)
print('\nFilling 1')
print(s)

print('\nFill 2')
s = fill2(s)
print(s)

print('\nEmpty 2')
s = empty2(s)
print(s)

print('\nPour 1 into 2 and fill 2')
s = pour12_fill2(s)
print(s)

print('\nEmpty 1')
s = empty1(s)
print(s)

print('\nPour 2 into 1 and empty 2')
s = pour21_empty2(s)
print(s)

print('\nPour 1 into 2 and empty 1')
s = pour12_empty1(s)
print(s)


child_bucket_states(BucketState(0, 0))


# your code here



Filling 1
(4, 0)

Fill 2
(4, 3)

Empty 2
(4, 0)

Pour 1 into 2 and fill 2
(1, 3)

Empty 1
(0, 3)

Pour 2 into 1 and empty 2
(3, 0)

Pour 1 into 2 and empty 1
None


Finally, we need to define the goal condition:

In [5]:
def goal_bucket_state(state):
    if state.b1 == BucketState.IDEAL_BUCKET_1:
        return True
    return False
    

Test your goal condition:

In [6]:
# Test goal condition
print(goal_bucket_state(BucketState(2, 0)))
print(goal_bucket_state(BucketState(0, 2)))
print(goal_bucket_state(BucketState(2, 2)))
print(goal_bucket_state(BucketState(1, 2)))
print(goal_bucket_state(BucketState(2, 1)))

True
False
True
False
True


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [7]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        if self.parent is None:
            self.depth = 0
        else:
            self.depth = self.parent.depth + 1
            

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [8]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            child_node = TreeNode(state=state, parent=node)
            
            # link child node to its parent in the tree
            node.add_child(child_node)
            
            # enqueue the child node
            queue.append(child_node)
            

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [9]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [10]:
# Print the solution's steps
def print_solution(node):
    if node.parent is not None:
        print_solution(node.parent)
    print(node.state)

Now we can print the solution:

In [11]:
print_solution(goal)

(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [12]:
def depth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    visited = set([initial_state]) # To avoid loops

    while queue:
        node = queue.pop()
        if goal_state_func(node.state):
            return node

        for state in operators_func(node.state):
            if state not in visited:
                visited.add(state)
                child_node = TreeNode(state=state, parent=node)
                node.add_child(child_node)
                queue.append(child_node)

    return None
    

Test it on the two buckets problem.

In [13]:
goal = depth_first_search(BucketState(0,0),goal_bucket_state, child_bucket_states)
print_solution(goal)


(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)
(0, 2)
(2, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [14]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    root = TreeNode(initial_state)   # create the root node in the search tree

    # Inner recursive function
    def sub_dfs(node, depth):
        if goal_state_func(node.state):
            return node
        if depth == depth_limit:
            return None

        for state in operators_func(node.state):
            child_node = TreeNode(state=state, parent=node)
            result = sub_dfs(child_node, depth+1)
            if result is not None:
                return result

    # Call the inner function with the root node and initial depth
    return sub_dfs(root, 0)

Test it on the two buckets problem.

In [15]:
goal = depth_limited_search(BucketState(0,0),goal_bucket_state, child_bucket_states, 7)
print_solution(goal)

(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [16]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit):
    for depth in range(depth_limit):
        result = depth_limited_search(initial_state, goal_state_func, operators_func, depth)
        if result is not None:
            return result
    return None

Again, test it on the two buckets problem.

In [17]:
goal = iterative_deepening_search(BucketState(0,0),goal_bucket_state, child_bucket_states, 10)
print_solution(goal)

(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


## The Missionaries and Cannibals Problem 

<img src="https://allfish24.files.wordpress.com/2016/09/gambar-1.jpg">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [18]:
# your code here
